# Import Packages & Load Data

In [2]:
import os
import zipfile
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models

In [4]:
# Unzip the training images
zip_path = "/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip"
extract_path = "/kaggle/working/train"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Move images into cats/ and dogs/ folders
os.makedirs(f"{extract_path}/cats", exist_ok=True)
os.makedirs(f"{extract_path}/dogs", exist_ok=True)

image_dir = os.path.join(extract_path, "train")  # actual image folder inside zip
for fname in os.listdir(image_dir):
    src = os.path.join(image_dir, fname)
    if fname.startswith("cat"):
        shutil.move(src, os.path.join(extract_path, "cats", fname))
    elif fname.startswith("dog"):
        shutil.move(src, os.path.join(extract_path, "dogs", fname))

# Clean up leftover folder
shutil.rmtree(image_dir)


# Preprocess with ImageDataGenerator

In [9]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    extract_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    extract_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# CNN Model

In [ ]:
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # output is probability of dog
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
#Train the Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


In [ ]:
# Visualize Training Curves
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Model Accuracy Over Epochs")
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Model Loss Over Epochs")
plt.show()

# Tuning

In [ ]:
from tensorflow.keras import regularizers

model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dropout(0.5),  # 🔥 NEW: turns off 50% neurons
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]


In [ ]:

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,  
    callbacks=callbacks
)

In [ ]:
# Visualize Training Curves
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Model Accuracy Over Epochs")
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Model Loss Over Epochs")
plt.show()

In [ ]:
# Grab one batch from val_gen
val_imgs, val_labels = next(val_gen)  # val_imgs.shape = (32, 150, 150, 3)

# Predict probabilities
val_preds = model.predict(val_imgs).flatten()
import matplotlib.pyplot as plt

class_names = ['Cat', 'Dog']

plt.figure(figsize=(15, 10))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(val_imgs[i])
    plt.axis('off')
    
    true_label = class_names[int(val_labels[i])]
    pred_prob = val_preds[i]
    pred_label = class_names[int(pred_prob > 0.5)]
    
    plt.title(f"Pred: {pred_label}\nProb: {pred_prob:.2f}\nTrue: {true_label}")
plt.tight_layout()
plt.show()


# Pre-trained models

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (300, 300)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [12]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load base model with 300x300 input
base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=(300, 300, 3)
)
base_model.trainable = False  # freeze initially

# Add custom classification head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train head
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks
)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 525s 763ms/step - accuracy: 0.4942 - loss: 0.7017 - val_accuracy: 0.5000 - val_loss: 0.6947 - learning_rate: 0.0010
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 465s 738ms/step - accuracy: 0.5088 - loss: 0.6990 - val_accuracy: 0.5004 - val_loss: 0.7002 - learning_rate: 0.0010
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 469s 745ms/step - accuracy: 0.5117 - loss: 0.6981 - val_accuracy: 0.5000 - val_loss: 0.6926 - learning_rate: 0.0010
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 471s 747ms/step - accuracy: 0.5137 - loss: 0.6976 - val_accuracy: 0.5000 - val_loss: 0.6917 - learning_rate: 0.0010
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 465s 738ms/step - accuracy: 0.5180 - loss: 0.6954 - val_accuracy: 0.5080 - val_loss: 0.6911 - learning_rate: 0.0010


In [7]:
# Unfreeze top layers
base_model.trainable = True

# Freeze earlier layers, fine-tune last ~30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Re-compile with lower learning rate
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune entire model
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


# ResNet50

In [3]:
# STEP 1: Data Generator with Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2
)

train_gen = datagen.flow_from_directory(
    "/kaggle/working/train",   # Make sure cats/ and dogs/ are subfolders here
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
# STEP 2: Build ResNet50V2 model with custom head
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, models

base_model = ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze initial layers

# Custom head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)

# Final model
model = models.Model(inputs=base_model.input, outputs=output)

# STEP 3: Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
# STEP 4: Add callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]


I0000 00:00:1745306488.568490     815 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745306488.569118     815 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [5]:
# STEP 5: Train the classifier head
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=callbacks
)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1745306506.350836     881 service.cc:148] XLA service 0x78e88c012eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745306506.350872     881 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745306506.350875     881 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745306507.574838     881 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/625 ━━━━━━━━━━━━━━━━━━━━ 44s 71ms/step - accuracy: 0.5938 - loss: 0.7197   

I0000 00:00:1745306514.214300     881 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 ━━━━━━━━━━━━━━━━━━━━ 297s 443ms/step - accuracy: 0.9627 - loss: 0.1267 - val_accuracy: 0.9786 - val_loss: 0.0596 - learning_rate: 0.0010
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 273s 433ms/step - accuracy: 0.9763 - loss: 0.0634 - val_accuracy: 0.9784 - val_loss: 0.0592 - learning_rate: 0.0010
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 268s 425ms/step - accuracy: 0.9807 - loss: 0.0508 - val_accuracy: 0.9808 - val_loss: 0.0539 - learning_rate: 0.0010
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 269s 426ms/step - accuracy: 0.9814 - loss: 0.0473 - val_accuracy: 0.9774 - val_loss: 0.0689 - learning_rate: 0.0010
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step - accuracy: 0.9843 - loss: 0.0437
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
625/625 ━━━━━━━━━━━━━━━━━━━━ 268s 426ms/step - accuracy: 0.9843 - loss: 0.0437 - val_accuracy: 0.9808 - val_loss: 0.0593 - learning_rate: 0.0010


In [9]:
# STEP 6: Fine-tune the top layers of ResNet
base_model.trainable = True

# Optionally freeze earlier layers
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Re-compile with lower LR for fine-tuning
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Continue training
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 301s 445ms/step - accuracy: 0.9597 - loss: 0.1139 - val_accuracy: 0.9668 - val_loss: 0.0830 - learning_rate: 0.0010
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 272s 431ms/step - accuracy: 0.9772 - loss: 0.0647 - val_accuracy: 0.9758 - val_loss: 0.0656 - learning_rate: 0.0010
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 417ms/step - accuracy: 0.9794 - loss: 0.0545 - val_accuracy: 0.9774 - val_loss: 0.0557 - learning_rate: 0.0010


In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from tqdm import tqdm

# Path to test folder (after proper unzip)
test_dir = "/kaggle/working/test_flat"  # <- update this if different
test_files = sorted(os.listdir(test_dir))

X_test = []
ids = []

for fname in tqdm(test_files):
    img_path = os.path.join(test_dir, fname)
    img = image.load_img(img_path, target_size=(224, 224))  # ✅ match ResNet input
    img_array = image.img_to_array(img) / 255.0
    X_test.append(img_array)
    ids.append(int(fname.split('.')[0]))

X_test = np.array(X_test)


100%|██████████| 12500/12500 [00:17<00:00, 715.29it/s]


In [ ]:
# Predict all at once
preds = model.predict(X_test, batch_size=64).flatten()  # Binary probabilities
import pandas as pd

submission = pd.DataFrame({
    'id': ids,
    'label': preds
})

submission = submission.sort_values('id')  # Ensure order
submission.to_csv("submission.csv", index=False)


# Final (ResNet50V2 + EfficientNetB3)

In [17]:
# STEP 1: Imports
from tensorflow.keras.applications import ResNet50V2, EfficientNetB3
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# STEP 2: Data Generators
img_size = (224, 224)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2,
    shear_range=0.2
)

train_gen = datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    "/kaggle/working/train",
    target_size=img_size,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# STEP 3: Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)

# STEP 4: Build Base Model Function
def build_model(base_fn):
    base_model = base_fn(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss=BinaryCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
    return model

# STEP 5: Train ResNet50V2 (no fine-tuning)
model_resnet = build_model(ResNet50V2)
model_resnet.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[early_stop, reduce_lr])

# STEP 6: Train EfficientNetB3 (no fine-tuning)
model_effnet = build_model(EfficientNetB3)
model_effnet.fit(train_gen, validation_data=val_gen, epochs=5, callbacks=[early_stop, reduce_lr])


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 287s 434ms/step - accuracy: 0.9393 - loss: 0.3696 - val_accuracy: 0.9818 - val_loss: 0.2381 - learning_rate: 0.0010
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 264s 419ms/step - accuracy: 0.9809 - loss: 0.2462 - val_accuracy: 0.9826 - val_loss: 0.2359 - learning_rate: 0.0010
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.9813 - loss: 0.2445
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 417ms/step - accuracy: 0.9813 - loss: 0.2445 - val_accuracy: 0.9804 - val_loss: 0.2366 - learning_rate: 0.0010
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 416ms/step - accuracy: 0.9884 - loss: 0.2334 - val_accuracy: 0.9838 - val_loss: 0.2339 - learning_rate: 5.0000e-04
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 266s 421ms/step - accuracy: 0.9867 - loss: 0.2335 - val_accuracy: 0.9860 - val_loss: 0.2299 - learn

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL InternalError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/errors_impl.py(462): __init__
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/execute.py(53): quick_execute
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/context.py(1683): call_function
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(251): call_flat
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(216): call_preflattened
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py(1322): _call_flat
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/tracing_compilation.py(139): call_function
  /usr/local/lib/pyt

In [ ]:
shutil.rmtree("/kaggle/working/test_flat", ignore_errors=True)

import zipfile

# Extract directly to /kaggle/working/test_flat
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working")

# Rename from /test → /test_flat to make it clear
import os
os.rename("/kaggle/working/test", "/kaggle/working/test_flat")


# STEP 7: TTA + Ensemble Predictions
test_dir = "/kaggle/working/test_flat"
test_files = sorted(os.listdir(test_dir))
batch_size = 500
ids = []
ensemble_preds = []

for i in tqdm(range(0, len(test_files), batch_size)):
    batch_files = test_files[i:i+batch_size]
    batch_imgs = []
    batch_imgs_flipped = []
    batch_ids = []

    for fname in batch_files:
        path = os.path.join(test_dir, fname)
        img = image.load_img(path, target_size=img_size)
        arr = image.img_to_array(img) / 255.0
        flipped = np.fliplr(arr)

        batch_imgs.append(arr)
        batch_imgs_flipped.append(flipped)
        batch_ids.append(int(fname.split('.')[0]))

    batch_imgs = np.array(batch_imgs)
    batch_imgs_flipped = np.array(batch_imgs_flipped)

    resnet_orig = model_resnet.predict(batch_imgs, batch_size=64).flatten()
    resnet_flip = model_resnet.predict(batch_imgs_flipped, batch_size=64).flatten()

    effnet_orig = model_effnet.predict(batch_imgs, batch_size=64).flatten()
    effnet_flip = model_effnet.predict(batch_imgs_flipped, batch_size=64).flatten()

    resnet_tta = (resnet_orig + resnet_flip) / 2
    effnet_tta = (effnet_orig + effnet_flip) / 2

    ensemble_pred = (resnet_tta + effnet_tta) / 2
    ensemble_pred = np.clip(ensemble_pred ** 1.2, 1e-5, 1 - 1e-5)

    ids.extend(batch_ids)
    ensemble_preds.extend(ensemble_pred)

# STEP 8: Create Submission
submission = pd.DataFrame({'id': ids, 'label': ensemble_preds})
submission = submission.sort_values('id')
submission.to_csv("submission_ensemble2.csv", index=False)


# Submission

In [6]:
shutil.rmtree("/kaggle/working/test_flat", ignore_errors=True)


In [7]:
shutil.rmtree("/kaggle/working/test_flat", ignore_errors=True)

import zipfile

# Extract directly to /kaggle/working/test_flat
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working")

# Rename from /test → /test_flat to make it clear
import os
os.rename("/kaggle/working/test", "/kaggle/working/test_flat")


In [8]:
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Path to the test images (ensure this folder has all .jpgs flat)
test_dir = "/kaggle/working/test_flat"
test_files = sorted(os.listdir(test_dir))

# Setup
batch_size = 500  # Number of images to load per mini-batch
ids = []
preds = []

# Predict in batches
for i in tqdm(range(0, len(test_files), batch_size)):
    batch_files = test_files[i:i+batch_size]
    batch_imgs = []
    batch_ids = []

    for fname in batch_files:
        img_path = os.path.join(test_dir, fname)
        img = image.load_img(img_path, target_size=(224, 224))  # match model input
        img_array = image.img_to_array(img) / 255.0
        batch_imgs.append(img_array)
        batch_ids.append(int(fname.split('.')[0]))

    batch_array = np.array(batch_imgs)
    batch_preds = model.predict(batch_array, batch_size=64).flatten()

    ids.extend(batch_ids)
    preds.extend(batch_preds)


  0%|          | 0/25 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step 


  4%|▍         | 1/25 [00:24<09:38, 24.12s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step


  8%|▊         | 2/25 [00:26<04:24, 11.49s/it]

...

 92%|█████████▏| 23/25 [01:23<00:05,  2.75s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step


 96%|█████████▌| 24/25 [01:26<00:02,  2.73s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step


100%|██████████| 25/25 [01:28<00:00,  3.56s/it]


In [12]:
submission = pd.DataFrame({
    'id': ids,
    'label': preds
})
submission = submission.sort_values('id')
submission.to_csv("submission2.csv", index=False)


In [ ]:
test_dir = "/kaggle/working/test_flat"

from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

test_files = sorted(os.listdir(test_dir))

X_test = []
ids = []

for fname in tqdm(test_files):
    img_path = os.path.join(test_dir, fname)
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    X_test.append(img_array)
    ids.append(int(fname.split('.')[0]))

X_test = np.array(X_test)


In [ ]:

# Predict all at once
preds = model.predict(X_test, batch_size=64).flatten()

# Create submission
submission = pd.DataFrame({'id': ids, 'label': preds})
submission = submission.sort_values('id')
submission.to_csv("submission.csv", index=False)


In [ ]:
# Predict probabilities
preds_effnet = model_effnet.predict(X_test, batch_size=64).flatten()
preds_resnet = model_resnet.predict(X_test, batch_size=64).flatten()

# Average (simple ensemble)
final_preds = (preds_effnet + preds_resnet) / 2
